#### Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_path = 'C:\\tmp\\nturgb+d_skeletons\\'
# список отсутсвующих элементов
broken_files_path = 'C:\\tmp\\NTU_RGBD_samples_with_missing_skeletons.txt'

In [3]:
# количество людей выполняющих действия
training_subjects = list(range(0, 19))
# классы которые будем использовать для обучения
training_classes = [1, 3, 4, 7, 8, 10, 13, 18]
training_cameras = [1, 2, 3]

max_body_true = 1
max_body_kinect = 1

num_joint = 25
max_frame = 300

In [4]:
def read_data(data_path, broken_files_path):
    labels = []
    files = []
    action_classes = {}
    counter = 0
    files_counter = {}
              
    with open(broken_files_path, 'r') as f:
        broken_files = f.read().split("\n")

    raw_files = os.listdir(data_path)
    num_frames = 0

    for filename in raw_files:
        if filename not in broken_files:
            action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
            subject_id   = int(filename[filename.find('P') + 1:filename.find('P') + 4])
            camera_id    = int(filename[filename.find('C') + 1:filename.find('C') + 4])
            if action_class in training_classes and camera_id in training_cameras:  #and subject_id in training_subjects:
                if action_class in action_classes:
                    if files_counter[action_class] < 120:
                        files.append([filename,action_classes[action_class]])
                        files_counter[action_class] = files_counter[action_class] + 1
                else:
                    action_classes.update({action_class : counter})
                    files_counter.update({action_class : 1})
                    counter += 1
                    files.append([filename,action_classes[action_class]])
#                     labels.append([action_class])
    print("action classes: ", action_classes)
    print("action files: ", files_counter)
    
    return files, action_classes

In [5]:
def get_nonzero_std(s): 
    index = s.sum(-1).sum(-1) != 0  
    s = s[index]
    if len(s) != 0:
        s = s[:, :, 0].std() + s[:, :, 1].std() + s[:, :, 2].std()  
    else:
        s = 0
    return s

def read_skeleton_filter(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []
        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []

            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)

    return skeleton_sequence

def read_xyz(file, max_body=1, num_joint=25):
    seq_info = read_skeleton_filter(file)
    data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
    for n, f in enumerate(seq_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < num_joint:
                    data[m, n, j, :] = [v['x'], v['y'], v['z']]

                else:
                    pass

    return data

In [6]:
# В этой функции меняем количество фреймов, подаваемых на вход модели
def create_coords_blocks(test_file, chonk_len = 30):

    frame_counter = 0
    new_labels = []
    new_frames = []
    blocks = []
    
    test_frames = read_xyz(data_path + test_file[0])[0]
    label = test_file[1]
    slice_len = chonk_len * int(len(test_frames)/chonk_len)


    for index in range(len(test_frames[:slice_len])):
        frame_counter += 1
        new_frames.append(test_frames[index].flatten())
        if frame_counter == chonk_len:
            frame_counter = 0
            blocks.append(np.array(new_frames))
            new_labels = new_labels + [label]
            new_frames = []

    return blocks, new_labels

In [7]:
joints_framework = ['neck','nose','mid_hip','l_sho','l_elb','l_wri','l_hip','l_knee','l_ank',
                    'r_sho','r_elb','r_wri','r_hip','r_kne','r_ank','r_eye','l_eye','r_ear','l_ear']

joints_framework_in_work = ['nose','l_sho','l_elb','l_wri','r_sho','r_elb','r_wri',
                            'l_hip','l_knee','l_ank','r_hip','r_kne','r_ank','neck']
upper_joints_framework   = ['nose','l_sho','l_elb','l_wri','r_sho','r_elb','r_wri',
                            'l_hip','l_knee','l_ank','r_hip','r_kne','r_ank','neck']

SKELETON_EDGES = np.array([[11, 10], [10, 9], [9, 0], [0, 3], [3, 4], [4, 5], [0, 6], [6, 7], [7, 8], [0, 12],
                               [12, 13], [13, 14], [1, 14], [1, 15], [15, 16], [1, 17], [17, 18]])

In [8]:
bone_pairs = (
    (1, 2), (2, 21), (3, 21), (4, 3), (5, 21), (6, 5),
    (7, 6), (8, 7), (9, 21), (10, 9), (11, 10), (12, 11),
    (13, 1), (14, 13), (15, 14), (16, 15), (17, 1), (18, 17),
    (19, 18), (20, 19), (22, 23), (21, 21), (23, 8), (24, 25),(25, 12)
)

bone_pairs_in_work = (
    (1, 14), 
    (14, 2), (2, 3), (3, 4),
    (14, 5), (5, 6), (6, 7), 
    (14, 8), (8, 9), (9, 10),
    (14, 11), (11, 12), (12, 13))

joints_names = ['spinebase', 'spinemid', 'neck', 'head','l_sho', 'l_elb','l_wri','l_hand',
                'r_sho','r_elb', 'r_wri', 'r_hand','l_hip','l_knee','l_ank','l_fool','r_hip',
                'r_knee','r_ank','r_foot','spineshoulder','l_tip','l_thumb','r_tip','r_thunb']
joints_in_work = [ 'head','l_sho', 'l_elb','l_wri','r_sho','r_elb', 'r_wri', 'l_hip','l_knee',
                  'l_ank','r_hip','r_knee','r_ank','spineshoulder']
upper_joints = [ 'head','l_sho', 'l_elb','l_wri','r_sho','r_elb', 'r_wri', 'l_hip','l_knee','l_ank',
                'r_hip','r_knee','r_ank','spineshoulder']

In [9]:
##### список файлов с лейблами на каждый файл
working_files_with_labels, action_classes = read_data(data_path, broken_files_path)

action classes:  {1: 0, 3: 1, 4: 2, 7: 3, 8: 4, 10: 5, 13: 6, 18: 7}
action files:  {1: 120, 3: 120, 4: 120, 7: 120, 8: 120, 10: 120, 13: 120, 18: 120}


In [10]:
LABELS = {v: k for k, v in action_classes.items()}

In [11]:
LABELS

{0: 1, 1: 3, 2: 4, 3: 7, 4: 8, 5: 10, 6: 13, 7: 18}

In [12]:
len(working_files_with_labels)

960

In [13]:
data = []
labels = []
##########################################################################
numbers = {0: 0, 1 : 0, 2 : 0, 3 : 0, 4 :0, 5 :0, 6 :0, 7 :0} #####
##################################################################
for file in working_files_with_labels:
    frames_blocks, label = create_coords_blocks(file)
    if label != [] and numbers[label[0]] <= 150:
        numbers[label[0]] = numbers[label[0]] + len(label)
        data = data + frames_blocks
        labels = labels + label
data_np = np.asarray(data)
labels_np = np.asarray(labels)

data_sq = data_np.reshape(len(data_np), -1)
test_data = pd.DataFrame(data_sq)
test_labels = pd.DataFrame(labels_np)
test_data['labels'] = test_labels

#### ИТОГОВЫЙ ФАЙЛ

In [14]:
test_data.to_csv("test_skels_30.csv", index = False)